# Lesson 1: Your first agent with Amazon Bedrock

## Preparation 
<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> and other files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

In [1]:
# Before you start, please run the following code to set up your environment.
# This code will reset the environment (if needed) and prepare the resources for the lesson.
# It does this by quickly running through all the code from the previous lessons.

!sh ./ro_shared_data/reset.sh

import os

roleArn = os.environ['BEDROCKAGENTROLE']

Resetting environment (if nessesary)
Agent reset process completed.
Lambda reset process completed.
Guardrail reset process completed.
Environment reset complete.


## Start of the lesson

In [2]:
import boto3

In [3]:
bedrock_agent = boto3.client(service_name='bedrock-agent', region_name='us-west-2')

In [4]:
create_agent_response = bedrock_agent.create_agent(
    agentName='mugs-customer-support-agent',
    foundationModel='anthropic.claude-3-haiku-20240307-v1:0',
    instruction="""You are an advanced AI agent acting as a front line customer support agent.""",
    agentResourceRoleArn=roleArn
)

In [5]:
create_agent_response

{'ResponseMetadata': {'RequestId': 'f091e16e-1fcd-4805-89ec-09102010459e',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Thu, 27 Mar 2025 16:44:57 GMT',
   'content-type': 'application/json',
   'content-length': '624',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'f091e16e-1fcd-4805-89ec-09102010459e',
   'x-amz-apigw-id': 'IGEFeFTePHcESow=',
   'x-amzn-trace-id': 'Root=1-67e58089-112be56a1b23e7e75826af02'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-west-2:730335477036:agent/Q6XERYX13U',
  'agentId': 'Q6XERYX13U',
  'agentName': 'mugs-customer-support-agent',
  'agentResourceRoleArn': 'arn:aws:iam::730335477036:role/c135316a3428926l9739562t1w73033547-BedrockAgentRole-y7ac7qslm8Fk',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2025, 3, 27, 16, 44, 57, 147667, tzinfo=tzlocal()),
  'foundationModel': 'anthropic.claude-3-haiku-20240307-v1:0',
  'idleSessionTTLInSeconds': 600,
  'instruction': 'You are an advanced AI agent acting as a

In [6]:
agentId = create_agent_response['agent']['agentId']

In [7]:
from helper import *

In [8]:
wait_for_agent_status(
    agentId=agentId, 
    targetStatus='NOT_PREPARED'
)

Waiting for agent status of 'NOT_PREPARED'...
Agent status: NOT_PREPARED
Agent reached 'NOT_PREPARED' status.


In [9]:
bedrock_agent.prepare_agent(
    agentId=agentId
)

{'ResponseMetadata': {'RequestId': '99c1e347-c2ff-4084-9c4d-d27655a95f2f',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Thu, 27 Mar 2025 16:45:16 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': '99c1e347-c2ff-4084-9c4d-d27655a95f2f',
   'x-amz-apigw-id': 'IGEIcEzlvHcEbng=',
   'x-amzn-trace-id': 'Root=1-67e5809c-722cd5d6289367206f4050c5'},
  'RetryAttempts': 0},
 'agentId': 'Q6XERYX13U',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2025, 3, 27, 16, 45, 16, 290168, tzinfo=tzlocal())}

In [10]:
wait_for_agent_status(
    agentId=agentId, 
    targetStatus='PREPARED'
)

Waiting for agent status of 'PREPARED'...
Agent status: PREPARED
Agent reached 'PREPARED' status.


In [11]:
create_agent_alias_response = bedrock_agent.create_agent_alias(
    agentId=agentId,
    agentAliasName='MyAgentAlias',
)

agentAliasId = create_agent_alias_response['agentAlias']['agentAliasId']

wait_for_agent_alias_status(
    agentId=agentId,
    agentAliasId=agentAliasId,
    targetStatus='PREPARED'
)

Waiting for agent alias status of 'PREPARED'...
Agent alias status: CREATING
Agent alias status: CREATING
Agent alias status: PREPARED
Agent alias reached status 'PREPARED'


In [12]:
bedrock_agent_runtime = boto3.client(service_name='bedrock-agent-runtime', region_name='us-west-2')

In [13]:
import uuid

In [14]:
message = "Hello, I bought a mug from your store yesterday, and it broke. I want to return it."

sessionId = str(uuid.uuid4())

invoke_agent_response = bedrock_agent_runtime.invoke_agent(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    endSession=False,
    enableTrace=True,
)

In [15]:
event_stream = invoke_agent_response["completion"]

In [16]:
for event in event_stream:
    print(event)

{'trace': {'agentAliasId': 'NZIDB1OANT', 'agentId': 'Q6XERYX13U', 'agentVersion': '1', 'sessionId': 'ae18c460-04ac-4196-b006-394f4a97f219', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'inferenceConfiguration': {'maximumLength': 2048, 'stopSequences': ['</invoke>', '</answer>', '</error>'], 'temperature': 0.0, 'topK': 250, 'topP': 1.0}, 'text': '{"system":" You are an advanced AI agent acting as a front line customer support agent. You have been provided with a set of functions to answer the user\'s question. You must call the functions in the format below: <function_calls>   <invoke>     <tool_name>$TOOL_NAME</tool_name>     <parameters>       <$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>       ...     </parameters>   </invoke> </function_calls> Here are the functions available: <functions>    </functions> You will ALWAYS follow the below guidelines when you are answering a question: <guidelines> - Think through the user\'s question, extract all data from the questi

{'trace': {'agentAliasId': 'NZIDB1OANT', 'agentId': 'Q6XERYX13U', 'agentVersion': '1', 'sessionId': 'ae18c460-04ac-4196-b006-394f4a97f219', 'trace': {'orchestrationTrace': {'modelInvocationOutput': {'metadata': {'usage': {'inputTokens': 1098, 'outputTokens': 86}}, 'rawResponse': {'content': '<thinking>I apologize, it seems there was an issue with my previous function call. Let me try a different approach to get the return policy information.</thinking>\n\n<function_calls>\n  <invoke>\n    <tool_name>GetReturnPolicy</tool_name>\n    <parameters>\n      <product>mug</product>\n    </parameters>'}, 'traceId': '554acadf-535f-45c5-abd3-b2f6b718648b-3'}}}}}
{'trace': {'agentAliasId': 'NZIDB1OANT', 'agentId': 'Q6XERYX13U', 'agentVersion': '1', 'sessionId': 'ae18c460-04ac-4196-b006-394f4a97f219', 'trace': {'orchestrationTrace': {'rationale': {'text': 'I apologize, it seems there was an issue with my previous function call. Let me try a different approach to get the return policy information.',

{'trace': {'agentAliasId': 'NZIDB1OANT', 'agentId': 'Q6XERYX13U', 'agentVersion': '1', 'sessionId': 'ae18c460-04ac-4196-b006-394f4a97f219', 'trace': {'orchestrationTrace': {'modelInvocationOutput': {'metadata': {'usage': {'inputTokens': 1586, 'outputTokens': 187}}, 'rawResponse': {'content': "I apologize, but it seems there may be an issue with the function calls I'm attempting to make. Let me try to assist you in a different way.\n\n<thinking>Since I'm unable to directly access the return policy information, let me try to gather some additional details from you about your purchase and the broken mug. This will help me provide the best possible guidance on how to proceed with the return.</thinking>\n\n<questions>\n- When and where did you purchase the mug?\n- Can you describe the issue with the mug, such as how it broke?\n- Do you have a receipt or any other proof of purchase?\n</questions>\n\nOnce I have those details, I'll do my best to advise you on the appropriate next steps for re

In [17]:
message = "Hello, I bought a mug from your store yesterday, and it broke. I want to return it."

sessionId = str(uuid.uuid4())

In [18]:
invoke_agent_and_print(
    agentAliasId=agentAliasId,
    agentId=agentId,
    sessionId=sessionId,
    inputText=message,
    enableTrace=True,
)

User: Hello, I bought a mug from your store yesterday, and it broke. I want
to return it.

Agent: 
Agent's thought process:
  Okay, let's see how I can assist you with returning the mug you
  purchased. I will need to gather some more information to process
  the return properly.

Agent's thought process:
  To process your return, I will need some more details about your
  order. Could you please provide the order ID or the date you made
  the purchase?

Observation:
  Type: FINISH

Final response:
  I apologize, but I seem to be having trouble retrieving the details
  of your order. Could you please provide me with the order ID or the
  date you made the purchase? That will help me look up the details
  and process your return request.


Session ID: 92aa59e8-47e1-402d-af38-746768d1f2ed
